In [1]:
!pip install opencv-python Pillow
!pip install opencv-python

In [2]:
import numpy as np
from scipy.ndimage import gaussian_filter

class FoveateVision:

    def __init__(self, fovy, sigma=1):
        self._cam_eye_fovy = fovy
        self._sigma = sigma

    def foveate(self, img, foveal_center=None):
        """
        Foveate the image, note that the shape of image has to be (height, width, 3)
        """

        # Define the foveal region
        fov = self._cam_eye_fovy
        foveal_size = 15
        foveal_pixels = int(foveal_size / 2 * img.shape[0] / fov)

        if foveal_center is None:  # default to center if not provided
            foveal_center = (img.shape[0] // 2, img.shape[1] // 2)

        # Create a circular mask for the foveal region
        y, x = np.ogrid[-foveal_center[0]:img.shape[0] - foveal_center[0], -foveal_center[1]:img.shape[1] - foveal_center[1]]
        mask = x ** 2 + y ** 2 <= (foveal_pixels ** 2)

        # Apply a Gaussian blur to each color channel separately
        blurred = np.zeros_like(img)
        for c in range(3):
            blurred_channel = gaussian_filter(img[:, :, c], sigma=self._sigma)
            blurred[:, :, c][~mask] = blurred_channel[~mask]

        # Combine the original image and the blurred image
        foveated = img.copy()
        foveated[~mask] = blurred[~mask]

        return foveated, foveal_center, foveal_pixels

In [8]:
import cv2

# Plot the close one

# Load the image using OpenCV
img_cv = cv2.imread('readclose.jpg')

# Convert the color space from BGR (default in OpenCV) to RGB
img_rgb = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)

# Now, img_rgb is your image array ready for processing
img = img_rgb

processor = FoveateVision(fovy=50, sigma=10)
result, foveal_center, foveal_pixels = processor.foveate(img, foveal_center=(400, 540))

# Draw a highlight circle around the fovea center using the returned foveal_center
highlight_color = (39, 78, 19)  # RGB: White
highlight_thickness = 2  # Thickness of 2 pixels
cv2.circle(result, (foveal_center[-1], foveal_center[0]), foveal_pixels, highlight_color, highlight_thickness)

# Convert the result back to BGR color space for saving with OpenCV
result_bgr = cv2.cvtColor(result, cv2.COLOR_RGB2BGR)

# Save the result using OpenCV
cv2.imwrite('vision_close.png', result_bgr)

True

In [9]:
# Plot the far one

# Load the image using OpenCV
img_cv = cv2.imread('readfar.jpg')

# Convert the color space from BGR (default in OpenCV) to RGB
img_rgb = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)

# Now, img_rgb is your image array ready for processing
img = img_rgb

processor = FoveateVision(fovy=75, sigma=10)
result, foveal_center, foveal_pixels = processor.foveate(img, foveal_center=(145, 150))

# Draw a highlight circle around the fovea center using the returned foveal_center
highlight_color = (0, 0, 255)  # RGB: White
highlight_thickness = 2  # Thickness of 2 pixels
cv2.circle(result, (foveal_center[-1], foveal_center[0]), foveal_pixels, highlight_color, highlight_thickness)

# Convert the result back to BGR color space for saving with OpenCV
result_bgr = cv2.cvtColor(result, cv2.COLOR_RGB2BGR)

# Save the result using OpenCV
cv2.imwrite('vision_far.png', result_bgr)

True

In [43]:
import cv2
import numpy as np

def draw_projected_arrow(img, start_point, end_point, arrowhead_length=30):
    # Draw main shaft of the arrow
    cv2.line(img, start_point, end_point, (255, 255, 255), thickness=3)
    
    # Calculate angle of the arrow direction
    angle = np.arctan2(start_point[1] - end_point[1], start_point[0] - end_point[0])
    
    # Calculate points for the arrowhead
    x1 = int(end_point[0] + arrowhead_length * np.cos(angle + np.pi/4))
    y1 = int(end_point[1] + arrowhead_length * np.sin(angle + np.pi/4))
    
    x2 = int(end_point[0] + arrowhead_length * np.cos(angle - np.pi/4))
    y2 = int(end_point[1] + arrowhead_length * np.sin(angle - np.pi/4))
    
    # Convert the points to int32
    arrowhead_pts = np.array([end_point, (x1, y1), (x2, y2)], dtype=np.int32)
    
    # Draw arrowhead
    cv2.polylines(img, [arrowhead_pts], isClosed=True, color=(255, 255, 255), thickness=3)

# Load image
img_cv = cv2.imread('readfar.jpg')
img_rgb = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)
img = img_rgb

# Define start and end points for arrow
start = (50, 300)
end = (400, 100)

# Draw projected arrow
draw_projected_arrow(img, start, end)

# Convert back to BGR and save
result_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
cv2.imwrite('arrow_projection.png', result_bgr)

True